In [3]:
import element_stiffness_linear_elastic_1D as es1D


In [4]:
fcn = es1D.element_stiffness_linear_elastic_1D

try:
    es1D.test_element_stiffness_comprehensive(fcn)
    print("Test passed!")
except AssertionError as e:
    print("Test failed:")
    print(e)

Test passed!


In [5]:
import numpy as np


def gauss_quadrature_1D(n: int) -> tuple[np.ndarray, np.ndarray]:
    """
    Return Gauss points and weights for 1D quadrature over [-1, 1].

    Parameters:
        n (int): Number of Gauss points (1, 2, or 3 recommended)

    Returns:
        tuple[np.ndarray, np.ndarray]: (points, weights)
    """
    if n == 1:
        points = np.array([0.0])
        weights = np.array([2.0])
    elif n == 2:
        sqrt_3_inv = 1.0 / np.sqrt(3)
        points = np.array([-sqrt_3_inv, sqrt_3_inv])
        weights = np.array([1.0, 1.0])
    elif n == 3:
        points = np.array([
            -np.sqrt(3/5), 0.0, np.sqrt(3/5)
        ])
        weights = np.array([
            5/9, 8/9, 5/9
        ])
    else:
        raise ValueError("Only 1 to 3 Gauss points are supported.")
    
    return points, weights


def shape_function_derivatives_1D_linear() -> np.ndarray:
    """
    Return constant derivatives of 1D linear shape functions w.r.t. ξ.

    Returns:
        np.ndarray: Derivatives [dN1/dξ, dN2/dξ]
    """
    return np.array([
        -0.5,
        0.5
    ])


def compute_jacobian_1D(dN_dxi: np.ndarray, x_elem: np.ndarray) -> float:
    """
    Compute the Jacobian for 1D isoparametric mapping.

    Parameters:
        dN_dxi (np.ndarray): Shape function derivatives w.r.t. ξ
        x_elem (np.ndarray): Nodal coordinates of the current element [x1, x2]

    Returns:
        float: Jacobian dx/dξ
    """
    return np.dot(dN_dxi, x_elem)

def element_stiffness_linear_elastic_1D(x_elem: np.ndarray, E: float, A: float, n_gauss: int) -> np.ndarray:
   """
   Compute the element stiffness matrix for a 1D linear bar using the Galerkin method.

   Parameters:
       x_elem (np.ndarray): Nodal coordinates of the element [x1, x2]
       E (float): Young's modulus
       A (float): Cross-sectional area
       n_gauss (int): Number of Gauss integration points (default = 2)

   Returns:
       np.ndarray: 2x2 element stiffness matrix
   """
   points, weights = gauss_quadrature_1D(n_gauss)
   dN_dxi = shape_function_derivatives_1D_linear()
   
   K = np.zeros((2, 2))
   
   for i in range(len(points)):
       J = compute_jacobian_1D(dN_dxi, x_elem)
       dN_dx = dN_dxi / J
       
       B = dN_dx.reshape(1, -1)
       K += E * A * np.dot(B.T, B) * weights[i] * J
   
   return K

In [6]:
fcn = element_stiffness_linear_elastic_1D

try:
    es1D.test_element_stiffness_comprehensive(fcn)
    print("Test passed!")
except AssertionError as e:
    print("Test failed:")
    print(e)


Test passed!


In [23]:
def test_element_stiffness_comprehensive(fcn):
    """Verify the correctness and robustness of the 1D linear elastic element stiffness matrix.

    This test checks the following properties of the stiffness matrix computed by
    `element_stiffness_linear_elastic_1D` for a two-node linear element:

    1. Analytical correctness:
        - For an element of length L with modulus E and area A, the stiffness matrix should match:
          (EA/L) * [[1, -1], [-1, 1]]

    2. Shape and symmetry:
        - The stiffness matrix must be 2x2 and symmetric.

    3. Singularity:
        - The matrix should be singular (zero determinant) for an unconstrained single element,
          reflecting rigid body motion.

    4. Integration consistency:
        - The result must be identical (within numerical tolerance) for 1-, 2-, and 3-point
          Gauss quadrature rules when applied to linear elements, since exact integration is achieved.
    """
    # Test parameters
    x_elem = np.array([0.0, 2.0])
    E = 200e9  # Young's modulus in Pa
    A = 0.01   # Cross-sectional area in m^2
    L = x_elem[1] - x_elem[0]  # Element length
    
    # Compute stiffness matrix with default Gauss points
    K = fcn(x_elem, E, A, 2)
    
    # 1. Test shape
    assert K.shape == (2, 2), f"Expected shape (2, 2), got {K.shape}"
    
    # 2. Test symmetry
    assert np.allclose(K, K.T), "Stiffness matrix must be symmetric"
    
    # 3. Test analytical correctness
    K_analytical = (E * A / L) * np.array([[1, -1], [-1, 1]])
    assert np.allclose(K, K_analytical), "Stiffness matrix does not match analytical solution"
    
    # 4. Test singularity (zero determinant for unconstrained element)
    det_K = np.linalg.det(K)
    assert abs(det_K) < 1e-10, f"Stiffness matrix should be singular, determinant = {det_K}"
    
    # 5. Test integration consistency across different Gauss point counts
    K_1gauss = fcn(x_elem, E, A, 1)
    K_2gauss = fcn(x_elem, E, A, 2)
    K_3gauss = fcn(x_elem, E, A, 3)
    
    assert np.allclose(K_1gauss, K_2gauss), "1-point and 2-point Gauss integration should give identical results"
    assert np.allclose(K_2gauss, K_3gauss), "2-point and 3-point Gauss integration should give identical results"
    
    # 6. Test with different element lengths
    x_elem_short = np.array([1.0, 1.5])
    L_short = x_elem_short[1] - x_elem_short[0]
    K_short = fcn(x_elem_short, E, A, 2)
    K_short_analytical = (E * A / L_short) * np.array([[1, -1], [-1, 1]])
    assert np.allclose(K_short, K_short_analytical), "Stiffness matrix incorrect for different element length"
    
    # 7. Test with different material properties
    E_steel = 210e9
    A_large = 0.02
    K_steel = fcn(x_elem, E_steel, A_large, 2)
    K_steel_analytical = (E_steel * A_large / L) * np.array([[1, -1], [-1, 1]])
    assert np.allclose(K_steel, K_steel_analytical), "Stiffness matrix incorrect for different material properties"
    
    # 8. Test positive definiteness properties
    eigenvals = np.linalg.eigvals(K)
    eigenvals_sorted = np.sort(eigenvals)
    print(eigenvals_sorted[0])
    print(K)
    assert abs(eigenvals_sorted[0]) < 1e-10, "First eigenvalue should be zero (rigid body mode)"
    assert eigenvals_sorted[1] > 0, "Second eigenvalue should be positive"
    
    # 9. Test row sum property (equilibrium condition)
    row_sums = np.sum(K, axis=1)
    assert np.allclose(row_sums, 0), "Row sums should be zero for equilibrium"
    
    # 10. Test column sum property (equilibrium condition)
    col_sums = np.sum(K, axis=0)
    assert np.allclose(col_sums, 0), "Column sums should be zero for equilibrium"

In [24]:
fcn = es1D.element_stiffness_linear_elastic_1D

try:
    test_element_stiffness_comprehensive(fcn)
    print("Test passed!")
except AssertionError as e:
    print("Test failed:")
    print(e)

-2.220446049250313e-07
[[ 1.e+09 -1.e+09]
 [-1.e+09  1.e+09]]
Test failed:
First eigenvalue should be zero (rigid body mode)


In [17]:
def test_element_stiffness_comprehensive(fcn):
    """
    Verify the correctness and robustness of the 1D linear elastic element stiffness matrix.
    """
    x_elem = np.array([0.0, 1.0])
    E = 210000000000.0
    A = 0.01
    L = x_elem[1] - x_elem[0]
    expected_stiffness = E * A / L * np.array([[1, -1], [-1, 1]])
    tolerance = 1e-09
    for n_gauss in [1, 2, 3]:
        stiffness_matrix = fcn(x_elem, E, A, n_gauss)
        assert np.allclose(stiffness_matrix, expected_stiffness, atol=tolerance), f'Stiffness matrix incorrect for n_gauss={n_gauss}'
        assert stiffness_matrix.shape == (2, 2), 'Stiffness matrix is not 2x2'
        assert np.allclose(stiffness_matrix, stiffness_matrix.T, atol=tolerance), 'Stiffness matrix is not symmetric'
        assert np.isclose(np.linalg.det(stiffness_matrix), 0, atol=tolerance), 'Stiffness matrix is not singular'

In [18]:
fcn = es1D.element_stiffness_linear_elastic_1D

try:
    test_element_stiffness_comprehensive(fcn)
    print("Test passed!")
except AssertionError as e:
    print("Test failed:")
    print(e)

Test passed!


In [21]:
def test_element_stiffness_comprehensive(fcn):
    """
    Verify the correctness and robustness of the 1D linear elastic element stiffness matrix.
    This test checks the following properties of the stiffness matrix computed by
    `element_stiffness_linear_elastic_1D` for a two-node linear element:
    1. Analytical correctness:
          (EA/L) * [[1, -1], [-1, 1]]
    2. Shape and symmetry:
    3. Singularity:
          reflecting rigid body motion.
    4. Integration consistency:
          Gauss quadrature rules when applied to linear elements, since exact integration is achieved.
    """
    x_elem = np.array([0.0, 1.0])
    E = 2.0
    A = 3.0
    L = x_elem[1] - x_elem[0]
    expected_matrix = E * A / L * np.array([[1, -1], [-1, 1]])
    computed_matrix = fcn(x_elem, E, A, n_gauss=2)
    print(computed_matrix)
    np.testing.assert_allclose(computed_matrix, expected_matrix, rtol=1e-10)
    assert computed_matrix.shape == (2, 2)
    np.testing.assert_allclose(computed_matrix, computed_matrix.T, rtol=1e-10)
    assert np.linalg.det(computed_matrix) < 1e-10
    matrix_1pt = fcn(x_elem, E, A, n_gauss=1)
    matrix_2pt = fcn(x_elem, E, A, n_gauss=2)
    matrix_3pt = fcn(x_elem, E, A, n_gauss=3)
    np.testing.assert_allclose(matrix_1pt, matrix_2pt, rtol=1e-10)
    np.testing.assert_allclose(matrix_1pt, matrix_3pt, rtol=1e-10)

In [22]:
fcn = es1D.element_stiffness_linear_elastic_1D

try:
    test_element_stiffness_comprehensive(fcn)
    print("Test passed!")
except AssertionError as e:
    print("Test failed:")
    print(e)

[[ 6. -6.]
 [-6.  6.]]
Test passed!
